In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;


In [3]:
#!import ../Utils/Settings.cs
#!import ../../../Plugins/CustomPlugin/CompanySearchPlugin.cs

In [4]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [5]:
Kernel kernel = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(Settings.AOAIModel , Settings.AOAIEndpoint, Settings.AOAIKey)
            .Build();

In [6]:
var companySearchPluginObj = new CompanySearchPlugin();
var companySearchPlugin = kernel.ImportPluginFromObject(companySearchPluginObj, "CompanySearchPlugin");

In [7]:
var pluginDirectory = Path.Combine("../../..", "plugins");

In [8]:
var writetPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "WriterPlugin"));
var emailPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "EmailPlugin"));
var translatePlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "TranslatePlugin"));

In [9]:
string goal =  """
Check the weather in Guangzhou, use spanish to write emails abount dressing tips based on the results
""";

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [16]:
#pragma warning disable SKEXP0060

var originalPlan = await planner.CreatePlanAsync(kernel, goal);

In [17]:
Console.WriteLine(originalPlan);

{{!-- Step 1: Check the weather in Guangzhou --}}
{{set "guangzhouWeather" (CompanySearchPlugin-WeatherSearch city="Guangzhou")}}

{{!-- Step 2: Write Spanish dressing tips based on the weather --}}
{{set "dressingTipsInSpanish" (WriterPlugin-Tips input=guangzhouWeather language="Spanish")}}

{{!-- Step 3: Write an email about the weather and dressing tips --}}
{{set "emailText" (EmailPlugin-WeatherMail input=dressingTipsInSpanish)}}

{{!-- Step 4: Output the result --}}
{{json emailText}}


In [18]:
originalPlan.ToString()

{{!-- Step 1: Check the weather in Guangzhou --}}
{{set "guangzhouWeather" (CompanySearchPlugin-WeatherSearch city="Guangzhou")}}

{{!-- Step 2: Write Spanish dressing tips based on the weather --}}
{{set "dressingTipsInSpanish" (WriterPlugin-Tips input=guangzhouWeather language="Spanish")}}

{{!-- Step 3: Write an email about the weather and dressing tips --}}
{{set "emailText" (EmailPlugin-WeatherMail input=dressingTipsInSpanish)}}

{{!-- Step 4: Output the result --}}
{{json emailText}}

In [19]:
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

In [20]:
Console.WriteLine(originalPlanResult);

Subject: Guía para vestimenta y atención a condiciones de viaje en Guangzhou

Estimado equipo,

Espero que este mensaje los encuentre bien. Comparto algunos consejos para nuestro viaje a Guangzhou, dadas las condiciones climáticas actuales. La ciudad está experimentando un clima frío y lluvioso, por lo que se recomienda vestirse de manera apropiada para mantenernos calientes y secos durante nuestra estancia.

Consideren vestirse con múltiples capas para mantener el calor. Sería aconsejable ponerse un abrigo o chaqueta impermeable que cubra materiales cálidos como la lana o el vellón. Si es posible, opten por ropa interior térmica para protegerse del frío. Elegir botas impermeables puede ser una buena opción para mantener nuestros pies secos y protegidos. Además, no olviden sus artículos de lluvia, como paraguas y una bolsa resistente al agua.

Es crucial que siempre estemos atentos a las condiciones de viaje, seguridad y clima en todo momento durante nuestro viaje. Teniendo en cuenta e